In [1]:
import math
import numpy as np
import pandas as pd

from datetime import datetime

import matplotlib.pyplot as plt
from scipy.integrate import odeint
import statsmodels.api as sm

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

plt.style.use('ggplot')
font_size = 14
plt.rcParams.update({'font.size': font_size})

In [2]:
def open_merge_files(infection_file_path, df_population):
    df_usa_rates = pd.read_csv(infection_file_path)
    df_usa_rates['datetime'] = pd.to_datetime(df_usa_rates['date'].astype(str), format='%Y%m%d')
    df_usa_rates['d_o_y'] = pd.DatetimeIndex(df_usa_rates['datetime']).dayofyear
    
    df_usa = pd.merge(df_usa_rates, df_population, how='outer', left_on='state', right_on='ABBR',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)[['datetime','d_o_y','NAME', 'state', 'POPESTIMATE2019','positiveIncrease', 
                         'positive', 'negativeIncrease', 'negative', 'pending', 'deathIncrease', 'death', 'recovered', 
                         'hospitalizedIncrease','hospitalized','totalTestResultsIncrease',
                         'totalTestResults','posNeg','total']]
    df_usa.columns = ['datetime','d_o_y','state_name', 'state_id', 'state_pop_2019','positive_daily_incr', 
                  'positive_cum', 'negative_daily_incr', 'negative_cum', 'pending_daily','death_daily_incr', 
                  'death_cum', 'recovered_cum', 'hospitalized_daily_incr','hospitalized_cum',
                  'total_test_results_daily_incr','total_test_results_cum','test_pos_neg_cum','total_tests_cum']
    return df_usa

In [3]:
df_usa_pop = pd.read_csv('../data/us_state_population_2019.csv')

In [4]:
df_usa = open_merge_files('../data/us_states_covid19_daily.csv', df_usa_pop)

In [5]:
df_usa

,datetime,d_o_y,state_name,state_id,state_pop_2019,positive_daily_incr,positive_cum,negative_daily_incr,negative_cum,pending_daily,death_daily_incr,death_cum,recovered_cum,hospitalized_daily_incr,hospitalized_cum,total_test_results_daily_incr,total_test_results_cum,test_pos_neg_cum,total_tests_cum
0,2020-05-10,131.0,Alaska,AK,731545.0,1.0,379.0,914.0,26985.0,NaN,0.0,10.0,324.0,0.0,NaN,915.0,27364.0,27364.0,27364.0
1,2020-05-09,130.0,Alaska,AK,731545.0,1.0,378.0,975.0,26071.0,NaN,0.0,10.0,318.0,0.0,NaN,976.0,26449.0,26449.0,26449.0
2,2020-05-08,129.0,Alaska,AK,731545.0,3.0,377.0,1129.0,25096.0,NaN,0.0,10.0,305.0,0.0,NaN,1132.0,25473.0,25473.0,25473.0
3,2020-05-07,128.0,Alaska,AK,731545.0,2.0,374.0,684.0,23967.0,NaN,0.0,10.0,291.0,0.0,NaN,686.0,24341.0,24341.0,24341.0
4,2020-05-06,127.0,Alaska,AK,731545.0,1.0,372.0,962.0,23283.0,NaN,1.0,10.0,284.0,0.0,NaN,963.0,23655.0,23655.0,23655.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3709,2020-03-11,71.0,Wyoming,WY,578759.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0
3710,2020-03-10,70.0,Wyoming,WY,578759.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0
3711,2020-03-09,69.0,Wyoming,WY,578759.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0
3712,2020-03-08,68.0,Wyoming,WY,578759.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0
